In [1]:
!pip install langchain langchain-openai chromadb beautifulsoup4 requests

  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached beautifulsoup4-4.13.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp310-cp310-win_amd64.whl.metadata (262 bytes)
  Using cached fastapi-0.115.8-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached onnxruntime-1.20.1-cp310-cp310-win_amd64.whl.metadata (4.7 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached grpcio-1.70.0-cp310-cp310-win_amd64.whl.metadata (4.0 kB)
  Using cached bcrypt-4.2.1-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached typer-0.15.1-py3-none-any.whl.metadata (15 kB)
  Using cached mmh3-5.1.0-cp310-c

In [9]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력
os.environ["USER_AGENT"] = "MyLangChainApp/1.0"

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o",
)

In [10]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

# 웹 크롤링하여 텍스트 데이터 가져오기
url = "https://ko.wikipedia.org/wiki/%EA%B2%80%EC%83%89%EC%A6%9D%EA%B0%95%EC%83%9D%EC%84%B1"  # AI 관련 위키 문서
loader = WebBaseLoader(url) # 웹 페이지의 내용을 가져와 랭체인 문서 객체로 변환
documents = loader.load() # 웹 페이지에서 텍스트 데이터를 추출

# 텍스트를 작은 단위로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # 웹에서 가져온 문서(documents)를 500자 단위로 분할
split_docs = text_splitter.split_documents(documents)

# OpenAI Embeddings로 벡터 변환
embeddings = OpenAIEmbeddings()

# ChromaDB 벡터 저장소 생성 및 데이터 저장
db = Chroma.from_documents(split_docs, embeddings, persist_directory="./chroma_db") # 분할된 문서를 벡터화하여 ChromaDB에 저장, 재사용 가능하도록 로컬 디스크에 저장

# ChromaDB에서 검색할 retriever 생성
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # 유사한 문서를 검색, 최대 5개의 유사한 검색 결과 반환

# LangChain Q&A 체인 설정
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff") # 검색된 문서를 기반으로 LLM이 답변 생성

# 사용자 질문 입력, 결과를 반환을 위한 검색 및 답변 생성
query = "RAG란?"
response = qa_chain.invoke(query)

print("📌 답변:")
print(response)

📌 답변:
{'query': 'RAG란?', 'result': 'RAG(검색증강생성, Retrieval-Augmented Generation)은 대형 언어 모델(LLM)의 성능을 향상시키기 위해 외부 데이터베이스나 문서 세트를 검색해서 사용하는 기술입니다. 즉, 모델이 내부의 방대한 정적 교육 데이터를 넘어서, 검색된 최신의 혹은 도메인별 정보를 참고하여 사용자 쿼리에 응답할 수 있도록 도와줍니다. 이 기술은 예를 들어 회사 내부 데이터에 기반한 챗봇 제공이나 신뢰할 수 있는 출처의 사실 정보 제공에 활용될 수 있습니다.'}


In [12]:
query = "RAG 사용 사례는?"
response = qa_chain.invoke(query)

print("📌 답변:")
print(response)

📌 답변:
{'query': 'RAG 사용 사례는?', 'result': 'RAG의 사용 사례에는 다음과 같은 예들이 있습니다:\n\n• 회사 내부 데이터에 접근하여 챗봇을 구축하는 경우  \n• 신뢰할 수 있는 소스에서 가져온 정보를 기반으로 사실 정보를 제공하는 경우\n\n이와 같이 RAG는 도메인별 및 최신 정보를 활용할 수 있도록 하여, 보다 정확하고 업데이트 된 결과를 제공하는 데 사용됩니다.'}
